<a href="https://colab.research.google.com/github/sophie-myk/AIFFEL_quest_cr/blob/main/Python/Deep_Learning_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
#입력데이터
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 *28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") /255

In [59]:
#모델
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
     layers.Dense( 512, activation= "relu"),
     layers.Dense(10, activation= "softmax")
])

In [60]:
#모델을 컴파일하는 단계
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy", #손실함수
              metrics=["accuracy"])

In [61]:
#마지막으로, 훈련반복
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8721 - loss: 0.4438
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9652 - loss: 0.1176
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9778 - loss: 0.0726
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9843 - loss: 0.0507
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9896 - loss: 0.0363


In [62]:
#1 첫번째 예제를 다시 밑바닥부터 구현
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        # Initialize weights
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        # Initialize biases
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

In [63]:
from re import X
class NaiveSequential:
  def __init__(self,layers):
      self.layers = layers

  def __call__(self,inputs):
      x = inputs
      for layer in self.layers:
        x = layer(x)
      return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
        weights += layer.weights
    return weights

In [64]:
model = NaiveSequential([
    NaiveDense(input_size= 28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size= 512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [65]:
#배치 제너레이터
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size:128):
      assert len(images) == len(labels)
      self.index = 0
      self.images = images
      self.labels = labels
      self.batch_size = batch_size
      self.num_batches = math.ceil(len(images)/batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images,labels

In [76]:
#훈련스텝 실행하기

def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
      predictions = model(images_batch)
      per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
      average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [77]:
learning_rate = 1e-3

def update_weights(gradients, weights):
  for g, w in zip(gradients, weights):
    w.assign_sub(g * learning_rate)

In [82]:
from tensorflow import optimizers

# Change the name to 'optimizer' to avoid confusion
optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    # Now you can use 'optimizer' here
    optimizer.apply_gradients(zip(gradients, weights))

In [83]:
#전체 훈련 루프_단순한 에포크의 반복전체 훈련 루프_단순한 에포크의 반복
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"에포크 {epoch_counter}")
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 손실: {loss:.2f}")

In [84]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 *28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") /255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
0번째 손실: 6.13
100번째 손실: 2.23
200번째 손실: 2.20
300번째 손실: 2.07
400번째 손실: 2.20
에포크 1
0번째 손실: 1.85
100번째 손실: 1.86
200번째 손실: 1.82
300번째 손실: 1.69
400번째 손실: 1.80
에포크 2
0번째 손실: 1.53
100번째 손실: 1.56
200번째 손실: 1.49
300번째 손실: 1.41
400번째 손실: 1.48
에포크 3
0번째 손실: 1.28
100번째 손실: 1.33
200번째 손실: 1.23
300번째 손실: 1.20
400번째 손실: 1.24
에포크 4
0번째 손실: 1.09
100번째 손실: 1.15
200번째 손실: 1.04
300번째 손실: 1.04
400번째 손실: 1.08
에포크 5
0번째 손실: 0.95
100번째 손실: 1.01
200번째 손실: 0.90
300번째 손실: 0.93
400번째 손실: 0.97
에포크 6
0번째 손실: 0.85
100번째 손실: 0.91
200번째 손실: 0.80
300번째 손실: 0.84
400번째 손실: 0.88
에포크 7
0번째 손실: 0.77
100번째 손실: 0.82
200번째 손실: 0.72
300번째 손실: 0.77
400번째 손실: 0.82
에포크 8
0번째 손실: 0.71
100번째 손실: 0.76
200번째 손실: 0.66
300번째 손실: 0.71
400번째 손실: 0.77
에포크 9
0번째 손실: 0.67
100번째 손실: 0.70
200번째 손실: 0.61
300번째 손실: 0.67
400번째 손실: 0.73


In [88]:
#모델 평가하기
import numpy as np # Add this line to import numpy

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1) # Now np is defined and can be used
matches = predicted_labels == test_labels
print(f"정확도: {matches.mean():.2f}")

정확도: 0.81
